In [1]:
import numpy as np 
import pandas as pd
import pdb
import requests

from Classifications import *
from keys import *

In [2]:
df = pd.read_csv("Meteorite_Landings.csv")
df.columns = ['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year',
       'reclat', 'reclong', 'GeoLocation']

### Drop mass NaNs & 0.0

In [3]:
df = df[(df.mass > 0.0) & (df.mass.isna() == False)]
df.shape

(45566, 10)

## Classifications

In [4]:
df = Classifications.classify_subclasses(df)

In [5]:
df.major_classification.value_counts()

Chondrite        42032
Achondrite        2178
Iron              1041
Stony-Iron         311
uncategorized        4
Name: major_classification, dtype: int64

In [6]:
df.groupby('major_classification').mean()

,id,mass,reclat,reclong
major_classification,,,,
Achondrite,30896.449954,2832.684123,-24.225972,37.769330
Chondrite,26885.663114,1522.094561,-41.663357,64.302098
Iron,17997.889529,494262.887233,8.131944,-6.022987
Stony-Iron,28007.141479,66261.350830,-2.451259,37.426869
uncategorized,43302.500000,0.078250,-79.683330,159.750000


## Country Coordinates

In [ ]:
lat, long = (0.0, 0.0)
latlng = f"{lat},{long}"

req_str = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latlng}&key={keys['mapsAPI']}"

# resp = requests.get(req_str).json()
# print(resp)
# for r in resp['results']:
#     if "country" in r['types']:
#         print(r['address_components'][0]['long_name'], "\n")
#         break

In [7]:
geo_df = df[(df.reclat.isna() == False) 
            & (df.reclong.isna() == False) 
            & (df.GeoLocation != '(0.0, 0.0)') # ocean placeholder
            & (df.GeoLocation != '(-71.5, 35.66667)') # antarctica 1
            & (df.GeoLocation != '(-84.0, 168.0)') # antarctice 2
           ]

geo_df.reset_index(inplace=True)
geo_df.head()

,index,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation,major_classification,subclass_category
0,0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",Chondrite,L
1,1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",Chondrite,H
2,2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",Chondrite,E
3,3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",Achondrite,Acapulcoite
4,4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",Chondrite,L


In [8]:
antarctic_df = df[(df.GeoLocation == '(-71.5, 35.66667)') | (df.GeoLocation == '(-84.0, 168.0)')]
antarctic_df.groupby('major_classification').mean()

,id,mass,reclat,reclong
major_classification,,,,
Achondrite,26280.362245,187.689796,-73.285714,54.571431
Chondrite,24842.045527,76.233004,-76.453484,88.107550
Iron,26194.250000,2082.552500,-72.541667,46.694448
Stony-Iron,23465.714286,16.765714,-78.642857,111.285716


In [11]:
test = pd.read_csv("countries.csv")
test.shape
# geo_df.merge(test, on="id", how='outer')

(17062, 5)

In [28]:
# import math
# for i in range(math.ceil(geo_df.shape[0] / 1000)):
#     file = f"countries/countries-{i}.csv"
#     with open(file, "w") as f:
#         f.write("id, reclat, reclong, country, country_abrv\n")
    
    #     !type nul > {file}  
# my_str = f"countries/{i}.csv"
# !type nul > {my_str}

In [34]:
import csv
def get_countries(df, start_idx):
    countries = []
    for i in range(start_idx, df.shape[0]):
        
        file = f"countries/countries-{(i // 1000)}.csv"
        
        row_id, lat, lng = (df.id[i], df.reclat[i], df.reclong[i])
        
        req_str = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={keys['mapsAPI']}"

        resp = requests.get(req_str).json()

        for r in resp['results']:
            if "country" in r['types']:
                country = r['address_components'][0]['long_name']
                country_short_name = r['address_components'][0]['short_name']
        
        try:    
            with open(file, 'a') as countries:
                country_writer = csv.writer(countries, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

                country_writer.writerow([row_id, lat, lng, country, country_short_name])
        except:
            print(i)
            return
        


# UNCOMMENT TO RUN API CALLS
# get_countries(geo_df[17062:].reset_index())
get_countries(geo_df.reset_index(), start_idx=0)

In [ ]:
df_test['country'] = countries
df_test